In [3]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, Subset
import pickle
from collections import Counter
import itertools
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, auc
import random
import pennylane
import torch
import math
from math import sqrt
import torch.nn as nn
from tqdm import tqdm
import cmath
from torch import tensor, tensordot
import time

In [ ]:
class AnsatzSimulation():


    H = [[(1/math.sqrt(2)), (1/math.sqrt(2))], [(1/math.sqrt(2)), -(1/math.sqrt(2))]]

    hadamard_gate = tensor(H, dtype=torch.complex64)


    pauli_x_gate = tensor([[0.0, 1.0],[1.0 ,0.0]], dtype=torch.complex64)

    pauli_y_gate = tensor([[0.0, -1.0j], [1.0j, 0.0]],dtype=torch.complex64)
    pauli_z_gate = tensor([[1.0, 0.0], [0.0,-1.0]], dtype=torch.complex64)

    phase_gate = tensor([[1.0, 0.0], [0.0, 1.0j]], dtype=torch.complex64)
    t_gate = tensor([[1.0, 0.0], [0.0, cmath.exp(math.pi*1.0j/4)]], dtype=torch.complex64)
    cnot_gate = tensor([[[[1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j]],

         [[0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j]]],


        [[[0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j]],

         [[0.+0.j, 0.+0.j],
          [1.+0.j, 0.+0.j]]]], dtype=torch.complex64)
    
   

    non_parametrized_gates = {
        'pauli_x': pauli_x_gate,
        'pauli_y': pauli_y_gate,
        'pauli_z': pauli_z_gate,
        'phase': phase_gate,
        't': t_gate,
        'hadamard': hadamard_gate
    }

    def __init__(self, n_qubits: int):
        self.n_qubits = n_qubits

    # Could do the embedding changing the rotation gates for each angle

    def rx_gate(self, theta = math.pi/2):
        return tensor([[math.cos(theta/2), -1j*math.sin(theta/2)],
                            [-1j*math.sin(theta/2), math.cos(theta/2)]], dtype=torch.complex64)

    def ry_gate(self, theta = math.pi/2):
        return tensor([[math.cos(theta/2), -math.sin(theta/2)], [math.sin(theta/2), math.cos(theta/2)]], dtype=torch.complex64)

    def rz_gate(self, theta = math.pi/2):
        return tensor([[-cmath.exp(-1.0j*theta/2), 0.0], [0.0, cmath.exp(-1.0j*theta/2)]], dtype=torch.complex64)
    
    gate_operation = {
        'pauli_x': pauli_x_gate,
        'pauli_y': pauli_y_gate,
        'pauli_z': pauli_z_gate,
        'phase': phase_gate,
        't': t_gate,
        'hadamard': hadamard_gate
    }

    def rotation_states(self, patch_vector, rotation_gate):
        state_vector = rotation_gate(patch_vector[0])
        for index in range(0, len(patch_vector) - 1):
            state_vector = torch.kron(state_vector, rotation_gate(patch_vector[index+1]))

        return state_vector
       
    
    def rx_state(self, angle):
        return tensor([[math.cos(angle/2), -1j*math.sin(angle/2)],
                            [-1j*math.sin(angle/2), math.cos(angle/2)]], dtype=torch.complex64)

    def ry_state(self, angle):
        return tensor([[math.cos(angle/2), -math.sin(angle/2)], [math.sin(angle/2), math.cos(angle/2)]], dtype=torch.complex64)
    
    def rz_state(self, angle):
        return tensor([[-cmath.exp(-1.0j*angle/2), 0.0], [0.0, cmath.exp(-1.0j*angle/2)]], dtype=torch.complex64)
    
    rotation_function = {
        'rx': rx_state,
        'ry': ry_state,
        'rz': rz_state
    }
    
    def randomAngleEmbedding(self, angle_tensor):
        rotation_gate_options = ['rx', 'ry', 'rz']
        angle_count = 0
        eye_tensor = torch.zeros(2**self.n_qubits, dtype=torch.complex64)
        eye_tensor[0] = 1.0
        for angle in angle_tensor:
            random.seed(520)
            chosen_gate = random.choice(rotation_gate_options)
            if angle_count == 0:
                state_vector = self.rotation_function[chosen_gate](self, angle)
            else:
                new_vector = self.rotation_function[chosen_gate](self, angle)
                state_vector = torch.kron(state_vector, new_vector)
            angle_count+=1
            
        return state_vector @ eye_tensor

    def uniformAngleEmbedding(self, patch_vector, rotation_gate: str):
        eye_tensor = torch.zeros(2**self.n_qubits, dtype=torch.complex64)
        eye_tensor[0] = 1.0
        if rotation_gate == 'rx':
            return self.rotation_states(patch_vector, self.rx_gate) @ eye_tensor
        elif rotation_gate == 'ry':
            return self.rotation_states(patch_vector, self.ry_gate) @ eye_tensor

        elif rotation_gate == 'rz':
            return self.rotation_states(patch_vector, self.rz_gate)  @ eye_tensor
        

    # Instead of simulating one qubit gate per time just do kronecker product to create a layer of quantum gates
    # To link all layers just do matmul

    # Also for simulating a controlled not gate, just do the tensor product between dims=([2,3], [control, target])
    # Tensor product between gates of the same layer would be better tbh, so just do

    def simulate_one_qubit_gate(self, selected_gate: torch.Tensor, state_vector: torch.Tensor, selected_qubit:int) -> tensor:
        return tensordot(selected_gate, state_vector, dims=([1],[selected_qubit]))

    def simulate_cnot(self, state_vector: torch.Tensor, target_qubit: int, control_qubit: int) -> tensor:
        return tensordot(self.cnot_gate, state_vector, dims=([2,3],[control_qubit, target_qubit]))

    def simulate_rotation_gate(self, selected_gate, state_vector, selected_qubit, angle=math.pi/2):
        rotation_tensor = self.rotation_function[selected_gate](self, angle)
        return tensordot(rotation_tensor, state_vector, dims=([1],[selected_qubit]))

    # Fix pauli Z measurement
    def pauliZ_test(self, state_vector, qubit_index):
        bitmask = 1 << qubit_index
        conjugated_state_vector = torch.conj(state_vector)

        for index in range(0, 2**self.n_qubits):
            if index & bitmask != 0:
                state_vector[index] = -state_vector[index]

        return torch.sum(conjugated_state_vector * state_vector)
    
    def pauliZ_expectationValue(self, state_vector, qubit_index):
        indices = torch.arange(2**self.n_qubits)
        signs = 1 - 2*((indices >> qubit_index) & 1)  # +1 for 0, -1 for 1
        expectation = torch.sum(signs * torch.abs(state_vector)**2)
        
        return expectation


    def simulate_circuit(self, input: torch.Tensor, embedding_type: str, ansatz_chromosome: list, parameters: torch.Tensor, measure: bool):
        layer_count = 0
        angle_count = 0
        #print('Starting ansatz simulation...')
        #start_time = time.perf_counter()
        state_vector = self.uniformAngleEmbedding(input, embedding_type).view((2,)*self.n_qubits)
        
        for layer in ansatz_chromosome:
            #print(f'Starting simulation at layer #{layer_count}')
            qubit_count = 0
            cnot_stack = []
            for gate in layer:
                #print(f'Starting simulation at qubit {qubit_count}... ')
                if gate in self.gate_operation:
                    state_vector = self.simulate_one_qubit_gate(self.gate_operation[gate], state_vector, qubit_count)
                elif gate in self.rotation_function:
                   # random_angle = random.uniform(0.0, 1.5)*math.pi
                    state_vector = self.simulate_rotation_gate(gate, state_vector, qubit_count, parameters[angle_count])
                    angle_count+=1
                elif gate != 'empty':
                    if gate[0:4] == 'ctrl' or gate[0:4] == 'trgt':
                        if not cnot_stack:
                            cnot_stack.append([gate[0:4], qubit_count])
                        else:
                            if gate[0:4] == 'trgt':
                                target_qubit = qubit_count
                                gate_name, control_qubit = cnot_stack[-1]
            
                            if gate[0:4] == 'ctrl':
                                control_qubit = qubit_count
                                gate_name, target_qubit = cnot_stack[-1]

                            state_vector = self.simulate_cnot(state_vector, target_qubit, control_qubit)
                            cnot_stack.pop()

                #print(f'Simulation at qubit {qubit_count} done!')

                qubit_count+=1
            layer_count+=1

        #end_time = time.perf_counter()
        #print(f'Simulation processing time: {end_time - start_time}')
        if measure:   
            state_vector = state_vector.view(-1)
            return [self.pauliZ_expectationValue(state_vector, qubit_index) for qubit_index in range(self.n_qubits)]
        else:
            return state_vector.view(-1)

In [ ]:
class GeneticAlgorithm():
    def __init__(self, crossover_rate: float, mutation_rate: float, no_generations: int, population_size: int, sampling, fitness, crossover, mutation, duplicates):
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.no_generations = no_generations
        self.population_size = population_size

        self.Sampling = sampling
        self.Fitness = fitness
        self.Crossover = crossover
        self.Mutation = mutation
        self.eliminate_duplicates = duplicates

    ## Population deve ter dimensões (population_size, solution_size)

    def initialize_population(self):
        return self.Sampling._do(self.population_size)
  
    def define_fitness(self, population, iteration) -> np.array:
        return np.array([self.Fitness._evaluate(individual, iteration, population.index(individual)) for individual in population])

    def select_parents(self, population: list):

        random_generator = np.random.default_rng(seed=42)
        indices = random_generator.choice(len(population), 2, replace=False)
        parent_a = population[indices[0]]
        parent_b = population[indices[1]]
        
        if len(parent_a) < len(parent_b):
            parent_a, parent_b = parent_b, parent_a
        return parent_a, parent_b

    def choose_parent(self) -> int:
        options = [0, 1]
        return random.choice(options)
    
    def crossover(self, parent_a, parent_b):
        return self.Crossover._do(parent_a, parent_b)

    def mutate_offspring(self, offspring) -> np.array:
        return self.Mutation._do(offspring)
    
    ## Use fitness values for elitism or something


    def run_algorithm(self) -> tuple:
        iterations = 0
        old_population = self.initialize_population()
        iteration_axis = []
        best_chromossome_axis = []
        while iterations != self.no_generations:
            print(old_population)
            print(f"iteration #{iterations}")
            iteration_axis.append(iterations)
            old_fitness_values = self.define_fitness(old_population, iterations)
            new_population = []
            for individuals in range(int(self.population_size/2)):
                parent_a, parent_b = self.select_parents(old_population)
                offspring_a, offspring_b = self.mutate_offspring(self.crossover(parent_a, parent_b))
                new_population.append(offspring_a)
                new_population.append(offspring_b)
            
            iterations +=1

            old_best_individual_position = np.argmax(old_fitness_values)
            old_best_fitness_value = np.max(old_fitness_values)
            old_best_individual = old_population[old_best_individual_position]

            new_fitness_values = self.define_fitness(new_population, iterations)
            new_best_individual_position = np.argmax(new_fitness_values)
            new_best_fitness_value = np.max(new_fitness_values)
            new_best_individual = new_population[new_best_individual_position]
        
            current_best_individual_fitness = 0.0
            if new_best_fitness_value > old_best_fitness_value:
                current_best_individual_fitness = new_best_fitness_value
                current_best_individual = new_best_individual
            else:
                current_best_individual_fitness = old_best_fitness_value
                current_best_individual = old_best_individual

    
            result_tuple = (current_best_individual, current_best_individual_fitness)
            best_chromossome_axis.append(current_best_individual_fitness)
        
            old_best_individual_position = np.argmax(old_fitness_values)
            random_position = random.randint(0, self.population_size - 1)
            new_population[random_position] = old_population[old_best_individual_position]
            old_population = new_population
            
        
        return result_tuple, best_chromossome_axis, iteration_axis

In [ ]:
class QuanvLayer(nn.Module):
    def __init__(self, n_qubits, patch_size, chromosome, mode="frozen"):
        super().__init__()
        self.n_qubits = n_qubits
        self.patch_size = patch_size
        self.chromosome = chromosome
        
        gates = list(itertools.chain.from_iterable(chromosome))
        gate_count = Counter(gates)
        n_params = gate_count['rx_gate'] + gate_count['ry_gate'] + gate_count['rz_gate']
        if mode == "frozen":
            self.ansatz_params = torch.rand(n_params)*math.pi
        elif mode == "trainable":
            self.ansatz_params = nn.Parameter(torch.rand(n_params)*math.pi)
            
        self.qlayer = AnsatzSimulation(n_qubits)
        
    def multichannel_quanv_2d(self, x):
        
        outputs = [[[self.qlayer.simulate_circuit(patch, 'ry', self.chromosome, self.ansatz_params, True) for patch in patches] for patches in channel] for channel in x]
        n_images, n_channels, n_patches, patch_size = tensor(outputs).shape
        image_size = int((n_patches * patch_size) ** 0.5)
        
        return tensor(outputs).view(n_images, image_size, image_size, n_channels)
    
    def greyscale_quanv_2d(self, x):
        
        outputs = [[self.qlayer.simulate_circuit(patch, 'rx', self.chromosome, self.ansatz_params, True) for patch in patches] for patches in x]
        
        return tensor(outputs)

    def forward(self, x):
        if len(x.shape) == 4:
            return self.multichannel_quanv_2d(x)
        elif len(x.shape) == 3:
            return self.greyscale_quanv_2d(x)
        
    
 
class L2NormalizationLayer(nn.Module):
    def __init__(self, dim=1, eps=1e-12):
        super(L2NormalizationLayer, self).__init__()
        self.dim = dim
        self.eps = eps

    def forward(self, x):
        return F.normalize(x, p=2, dim=self.dim, eps=self.eps)

class HybridModel(nn.Module):
    def __init__(self, n_qubits, patch_size, chromosome, num_classes, input_size, mode='frozen'):
        super().__init__()
        self.quanv_layer = QuanvLayer(
            n_qubits=n_qubits,
            patch_size=patch_size,
            chromosome=chromosome,
            mode=mode
        )
        feature_size = input_size**2
        self.fc1 = nn.Linear(feature_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.norm = nn.LayerNorm(feature_size)
        #self.conv1 = nn.Conv2d(in_channels = 1, output_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=patch_size, stride=patch_size)
        self.fc = nn.Linear(feature_size, num_classes)
        self.l2norm = L2NormalizationLayer(dim=1)
        
    def forward(self, x):
        print("Passing through quanvolution layer...")
        start_time = time.perf_counter()
        x = self.quanv_layer.forward(x)
        end_time = time.perf_counter()
        print(f'Quanvolution processing time: {end_time - start_time}')  
        x = x.flatten(start_dim=1)
        #print(x.shape)
        x = self.l2norm(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    

def train_model(model, train_loader, num_epochs, optimizer, loss_fn, filepath):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        epoch_loss = 0.0
        total = 0
        
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for i, (inputs, labels) in progress_bar:
        
            optimizer.zero_grad()  # Zero out previous gradients
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)  # Calculate loss
            
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            loss.backward()  # Backpropagate to calculate gradients
            optimizer.step() # Update weights
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/(i+1):.4f}, Acc: {correct / total:.4f}")
            
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: Train Loss={epoch_loss:.4f}, Train Acc={epoch_acc:.4f}")
            # Print every 10 batches
            
    torch.save(model.state_dict(), filepath)  
    
def validate_model(model, dataloader, criterion, device):
    model.eval()    
    model.to(device)

    total_loss = 0.0
    all_labels = []
    all_predictions = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
#            images = images.to(device)
#            labels = labels.squeeze().to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy() if outputs.shape[1] > 1 else torch.softmax(outputs, dim=1)[:, 0].cpu().numpy()
            all_probs.extend(probs)
            all_predictions.extend(outputs.argmax(dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auc = float("nan")

    avg_loss = total_loss / len(dataloader)
    print(f"Validation Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    return avg_loss, accuracy, precision, recall, f1, auc

  

In [12]:
def swap_cnot_gate(circuit_layer: list, selected_qubit: int):
    """
    This function swaps the control and target qubits in 
    the CNOT gate

    """

    gate = circuit_layer[selected_qubit]
    if gate[0:4] == 'ctrl': # testar esse if
        cnot_value = gate[5]
        target_index = circuit_layer.index('trgt_'+ cnot_value)
        circuit_layer[selected_qubit], circuit_layer[target_index] =  circuit_layer[target_index], circuit_layer[selected_qubit] 
    elif gate[0:4] == 'trgt':
        cnot_value = gate[5]
        control_index = circuit_layer.index('ctrl_' + cnot_value)
        circuit_layer[selected_qubit], circuit_layer[control_index] =  circuit_layer[control_index], circuit_layer[selected_qubit]
    
    return circuit_layer

def change_cnot_gate_to_one_qubit_gates(circuit_layer: list, gate_options: list, selected_qubit: int) -> list:

    """
    This function turns the control and target qubits into one-qubit gates

    """

    gate = circuit_layer[selected_qubit]
    gate_index = gate[5]
    if gate[0:4] == 'ctrl':
        target_index = circuit_layer.index('trgt_' + gate_index)
        circuit_layer[control_index] = np.random.choice(gate_options)
    elif gate[0:4] == 'trgt':
        control_index = circuit_layer.index('ctrl_' + gate_index)
        circuit_layer[control_index] = np.random.choice(gate_options)
    
    circuit_layer[selected_qubit] = np.random.choice(gate_options)
    
    return circuit_layer

class AnsatzMutation:

    """
    This Mutation Class randomly swaps Ansatz layers and changes the  
    the gates placed in certain positions

    """
    def __init__(self, mutation_rate: float):
        self.mutation_rate = mutation_rate
        self.one_qubit_gates = ['empty', 'pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard']
        self.gate_options_with_cnot = ['pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard', 'ctrl', 'trgt']

    def _do(self, offspring):
        mutated_offspring = []
        for individual in offspring:
            mutated_individual = []
            for circuit_layer in individual:
                random_value = np.random.random()
                if random_value < self.mutation_rate:
                    random_qubit_position = np.random.randint(0, len(circuit_layer))
                    random_gate =  circuit_layer[random_qubit_position]
                    if random_gate[0:4] == 'ctrl' or random_gate[0:4] == 'trgt': 
                        random_mutation_action = np.random.randint(0,2)
                        random_qubit_position = np.random.randint(0, len(circuit_layer))
                        if random_mutation_action == 0:
                            mutated_individual.append(swap_cnot_gate(circuit_layer, random_qubit_position))
                        elif random_mutation_action == 1:
                            mutated_individual.append(change_cnot_gate_to_one_qubit_gates(circuit_layer, self.one_qubit_gates, random_qubit_position))
                    else:
                        new_layer = circuit_layer
                        new_layer[random_qubit_position] = random.choice(self.gate_options_with_cnot)
                        mutated_individual.append(new_layer)
                else:
                    mutated_individual.append(circuit_layer)
            mutated_offspring.append(mutated_individual)

        print(len(mutated_offspring))
        offspring_a, offspring_b = mutated_offspring
        return offspring_a, offspring_b
    
def select_parents(population: list):
    random_generator = np.random.default_rng(seed=42)
    parent_a, parent_b = random_generator.choice(a = np.array(population), size=2, replace=False)
    if parent_a.shape[1] < parent_b.shape[1]:
        parent_a, parent_b = parent_b, parent_a
    return parent_a, parent_b


# Gotta test AnsatzCrossover
class AnsatzCrossover:
    """"
    The crossover happens between chromosomes of possibly different lengths,
    then the offspring might inherit the size of one of its parents
    """

    def __init__(self, crossover_rate: float):
        self.crossover_rate = crossover_rate

    def _do(self, parent_a, parent_b):
        random_number = np.random.random()
        if random_number < self.crossover_rate:
            return [parent_a, parent_b]
        
        random_generator = np.random.default_rng(seed=42)

        # The parent b is always the smallest one, and the parent a, the biggest one

        chromosome_length_a = len(parent_b)
        chromosome_length_b = len(parent_a)

        crossover_point_a = random_generator.integers(low=2, high=chromosome_length_a, size=1)[0]
        crossover_point_b = random_generator.integers(low=2, high=chromosome_length_b,size=1)[0]

        # Could flip a coin to change which parent comes first
        flip_coin = random_generator.choice([0, 1])

        if flip_coin == 0:
            offspring_a = parent_a[0:crossover_point_a] + parent_b[crossover_point_b:]
            offspring_b = parent_b[0:crossover_point_b] + parent_a[crossover_point_a:]
        else:
            offspring_a = parent_b[0:crossover_point_b] + parent_a[crossover_point_a:]
            offspring_b = parent_a[0:crossover_point_a] + parent_b[crossover_point_b:]

    
        return [offspring_a, offspring_b]
    
class RemoveEquivalentAnsätze:

    def __init__(self, n_tests, threshold_value, n_qubits):
        self.n_tests = n_tests
        self.threshold_value = threshold_value
        self.n_qubits = n_qubits
        self.circuit_ansatz = AnsatzSimulation(self.n_qubits)
        
        

    def generate_random_parameters(self, ansatz):
        gates = list(itertools.chain.from_iterable(ansatz.tolist()))
        gate_count = Counter(gates)
        n_params = gate_count['rx_gate'] + gate_count['ry_gate'] + gate_count['rz_gate']
        parameters = torch.rand(n_params)*math.pi

        return parameters

    def test_ansatz(self, ansatz, parameters, state_vector):
        return self.circuit_ansatz.simulate_circuit(input=state_vector, embedding_type='rx', ansatz_chromosome=ansatz, parameters=parameters, measure=False)
            
    # Check whether i calculate the inner product with just the real parts or should i include the imaginary ones as well
   
    def fidelity(self, state_vector_psi, state_vector_phi):
        return 1 - torch.einsum('i,i->i',state_vector_psi, state_vector_phi)**2

    # Decide which measure should be used in order to evaluate the

    def is_equal(self, ansatz_a, ansatz_b):
        print(type(ansatz_a))
        ansatz_a = ansatz_a.tolist()
        ansatz_b = ansatz_b.tolist()
        input_test = torch.rand(self.n_tests, self.n_qubits)
        params_a = self.generate_random_parameters(ansatz_a)
        params_b = self.generate_random_parameters(ansatz_b)
        
        output_a = [self.test_ansatz(ansatz_a, params_a, state_vector) for state_vector in input_test]
        output_b = [self.test_ansatz(ansatz_b, params_b, state_vector) for state_vector in input_test]
        output_pair = list(zip(output_a, output_b))
        
        fidelity_score = torch([self.fidelity(state_psi, state_phi) for (state_psi, state_phi) in output_pair])
        
        score = torch.mean(fidelity_score)
        
        return self.threshold_value > score[0]

In [13]:
mutation_rate = 0.1
offsprings = [[['pauli_y', 'pauli_x', 'ctrl_0', 'trgt_0'], ['ry_gate', 'pauli_x', 'phase', 't'], ['ctrl_0', 'trgt_0', 'phase', 'pauli_z'], ['t', 'rx_gate', 'phase', 'pauli_z'], ['phase', 'pauli_x', 'ry_gate', 't']], [['pauli_z', 'trgt', 'pauli_z', 'pauli_y'], ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], ['pauli_x', 'ctrl_0', 'trgt_0', 'pauli_x'], ['ry_gate', 'ry_gate', 'rx_gate', 'rx_gate']]]
mutate = AnsatzMutation(mutation_rate)
mutate._do(offsprings)

2


([['pauli_y', 'pauli_x', 'ctrl_0', 'trgt_0'],
  ['ry_gate', 'pauli_x', 'phase', 't'],
  ['ctrl_0', 'trgt_0', 'phase', 'pauli_z'],
  ['t', 'rx_gate', 'phase', 'pauli_z'],
  ['phase', 'pauli_x', 'ry_gate', 't']],
 [['pauli_z', 'trgt', 'pauli_z', 'pauli_y'],
  ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'],
  ['pauli_x', 'ctrl_0', 'trgt_0', 'pauli_z'],
  ['ry_gate', 'ry_gate', 'rx_gate', 'rx_gate']])

In [ ]:
class QuantumCircuitOptimization:
    def __init__(self, n_qubits, n_gates, possible_gates, n_layers, patch_size, n_classes, input_size, mode, max_gates, dataset, batch_size, train_ratio):
        self.n_gates = n_gates,
        self.n_qubits = n_qubits
        self.n_layers = n_layers 
        self.possible_gates = possible_gates
        self.patch_size = patch_size
        self.n_classes = n_classes
        self.input_size = input_size 
        self.mode = mode
        self.max_gates = max_gates
        
        

        train_len=int(len(dataset)*train_ratio)
        test_len=len(dataset)-int(len(dataset)*train_ratio)
        train_set= Subset(dataset,range(0,train_len))
        val_set=Subset(dataset,range(train_len,len(dataset)))
                
        
        self.train_load = DataLoader(train_set, batch_size, shuffle=True)
        self.val_load = DataLoader(val_set, batch_size, shuffle=True)
        
    def _evaluate(self, x, generation, individual):

        # Amount of layers * # of qubits
        ansatz_depth = len(x) * self.n_qubits
        if ansatz_depth > self.max_gates:
            return max(0, ansatz_depth - self.max_gates)
        else:
            hqcnn = HybridModel(self.n_qubits, self.patch_size, x, self.n_classes, self.input_size, self.mode)
            optimizer = torch.optim.Adam(hqcnn.parameters(), lr=0.1, weight_decay=1e-4)
            loss_fn = nn.CrossEntropyLoss()

            device = torch.device("cpu")
            n_epochs = 5
            
            filepath = f'weights/mnist_hqcnn_GA_gen_#{generation}_individual_#{individual}'
            train_model(hqcnn, self.train_load, n_epochs, optimizer, loss_fn, filepath)
           
            val_hqcnn = HybridModel(self.n_qubits, self.patch_size, x, self.n_classes, self.input_size, self.mode)
            val_hqcnn.load_state_dict(torch.load(filepath, weights_only=True))
            avg_loss, accuracy, precision, recall, f1, auc = validate_model(hqcnn, self.val_load, loss_fn, device)
            return -f1

# Gotta test if sampling is working
class AnsatzRepair:

    def _do(self, X):
        return X


class QuantumCircuitSampling:
    
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
        self.quantum_gate_options = ['empty', 'pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard', 'ctrl', 'trgt']
        self.gate_options_without_cnot = ['pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard']
        self.rotation_gate_options = ['rx_gate', 'ry_gate', 'rz_gate']
        self.non_parametrized_gates = ['pauli_x', 'pauli_y','pauli_z', 'phase','t','hadamard', 'ctrl', 'trgt']
    
    def generate_layer_without_entanglement(self):
       return [random.choice(self.gate_options_without_cnot) for wire in range(self.n_qubits)]
    
    def generate_rotation_layer(self):
        return [random.choice(self.rotation_gate_options) for wire in range(self.n_qubits)]
    
    def place_a_single_gate(self):
        return random.choice(self.gate_options_without_cnot) 
    
    def generate_disjoint_cnots(self):
        cnot_count_layer_one = 0
        cnot_count_layer_two = 0
        layer_one = []
        layer_two = []
        
        layer_two.append(self.place_a_single_gate())
        
        for wire in range(self.n_qubits - 1):
            if wire%2 == 0:
                layer_one.append(f'ctrl_{cnot_count_layer_one}')
                layer_one.append(f'trgt_{cnot_count_layer_one}')
                cnot_count_layer_one += 1
               
            else:
                layer_two.append(f'ctrl_{cnot_count_layer_two}')
                layer_two.append(f'trgt_{cnot_count_layer_two}')
                cnot_count_layer_two += 1
            
        if self.n_qubits%2 == 0:
            layer_two.append(self.place_a_single_gate())
        else:
            layer_one.append(self.place_a_single_gate())
        
        return layer_one, layer_two
    
    def generate_non_parametrized_layer(self):
        return [random.choice(self.non_parametrized_gates) for wire in range(self.n_qubits)]

    def generate_partially_entangled_layer(self):
        pairs = [(wire,wire+1) for wire in range(self.n_qubits-1)]
        #print(pairs)
        selected_pair = random.choice(pairs)
        control, target = selected_pair
        layer = ['ctrl_0' if wire == control else
                'trgt_0' if wire == target else
                self.place_a_single_gate()
                for wire in range(self.n_qubits)]
        
        return layer
    
    def _do(self, n_samples):
        print('where are you?')
        population = []
        
        for individual in range(n_samples):
            print(f'Generating individual...')
            ansatz = []
            n_layers = random.randint(4, 10)
            print(f'Generating {n_layers} layers...')
            depth = 0
            while depth != n_layers:
                print(f'Generating layer #{depth}')
                gate_layer = []
                layer_type = random.randint(0,4)
                if layer_type == 0:
                    layer = self.generate_layer_without_entanglement()
                    ansatz.append(layer)
                    depth+=1
                elif layer_type == 1:
                    layer_one, layer_two = self.generate_disjoint_cnots()
                    ansatz.append(layer_one)
                    ansatz.append(layer_two) 
                    depth+=2
                elif layer_type == 2:
                    layer = self.generate_rotation_layer()
                    ansatz.append(layer)
                    depth+=1
                elif layer_type == 3:
                    layer = self.generate_non_parametrized_layer()
                    ansatz.append(layer)
                    depth+=1
                elif layer_type == 4:
                    layer = self.generate_partially_entangled_layer()
                    ansatz.append(layer)
                    depth+=1
            population.append(ansatz)
                           
        return population
    


In [ ]:
def sample_data(dataset, classes, sample_size):
    class_one = 0
    class_two = 0
    binary_dataset = []

    for image in tqdm(dataset, desc='sampling data'):
        if image[1] == classes[0] and class_one < sample_size:
            binary_dataset.append(image)
            class_one +=1 
        elif image[1] == classes[1] and class_two < sample_size:
            binary_dataset.append(image)
            class_two += 1
            
        if class_one == sample_size and class_two == sample_size:
            break
    
    return binary_dataset

class SampledDataset4Training(Dataset):
    def __init__(self, dataset, target_classes, transform=None):
        images, labels = zip(*dataset)
        labels = [0 if label == target_classes[0] else 1 for label in tqdm(labels, desc='labeling')]
        self.labels = tensor(labels)
        self.images = torch.stack(images)
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
            
        
        if self.transform:
            sample = self.transform(self.images[index])
        else:
            sample = self.images[index]
        
        
        return sample, self.labels[index]

class PatchExtraction(object):
    def __init__(self, patch_size: int):
        self.patch_size = patch_size
    
    def __call__(self, image):
    
        unfold = nn.Unfold(kernel_size=(self.patch_size, self.patch_size), stride=self.patch_size)
        image_patches = unfold(image)
        patch_len, n_patches = image_patches.shape
        image_patches = image_patches.view((n_patches, patch_len))
        return image_patches
        

In [2]:
population_size = 2
crossover_rate = 0.5
mutation_rate = 0.1
generations = 2
seed = 42

# Circuit ansatz hyperparameters
n_tests = 4
equivalence_ratio = 0.7
n_qubits = 4
max_gates = 32
gate_options = [None, 'pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard', 'ctrl', 'trgt']
max_layers = 8

# Model and training hyperparameters
input_size = 20
n_classes = 2
mode = 'frozen'
batch_size = 50
train_ratio = 0.7
patch_size = 2

In [ ]:

transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    PatchExtraction(patch_size)])

target_classes = [0, 1]
sample_size = 200

full_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


sampled_training_dataset = sample_data(full_trainset, target_classes, sample_size)
mnist_training_dataset = SampledDataset4Training(sampled_training_dataset, target_classes)


ansatz_optimization_problem = QuantumCircuitOptimization(n_qubits=n_qubits,
                                                            n_gates=max_gates,
                                                            possible_gates=gate_options,
                                                            n_layers=max_layers,
                                                            patch_size=patch_size,
                                                            max_gates=max_gates,
                                                            input_size=input_size,
                                                            n_classes=n_classes,
                                                            mode=mode,
                                                            dataset=mnist_training_dataset,
                                                            batch_size=batch_size,
                                                            train_ratio=train_ratio)

duplicate_ansatz = RemoveEquivalentAnsätze(n_tests, equivalence_ratio, n_qubits)

genetic_algo = GeneticAlgorithm(crossover_rate=crossover_rate,
                                mutation_rate=mutation_rate,
                                no_generations=generations,
                                population_size=population_size,
                                sampling=QuantumCircuitSampling(n_qubits),
                                fitness=ansatz_optimization_problem,
                                crossover=AnsatzCrossover(crossover_rate),
                                mutation=AnsatzMutation(mutation_rate),
                                duplicates=duplicate_ansatz)

result, best_solution_axis, iterations = genetic_algo.run_algorithm()


In [ ]:
offsprings = [[['pauli_y', 'pauli_x', 'ctrl_0', 'trgt_0'], ['ry_gate', 'pauli_x', 'phase', 't'], ['ctrl_0', 'trgt_0', 'phase', 'pauli_z'], ['t', 'rx_gate', 'phase', 'pauli_z'], ['phase', 'pauli_x', 'ry_gate', 't']], [['pauli_z', 'trgt', 'pauli_z', 'pauli_y'], ['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], ['pauli_x', 'ctrl_0', 'trgt_0', 'pauli_x'], ['ry_gate', 'ry_gate', 'rx_gate', 'rx_gate']]]
mutate = AnsatzMutation(mutation_rate)
mutate._do(offsprings)

In [9]:
mutate = AnsatzMutation(mutation_rate)

In [10]:
mutate._do(offsprings)

9


ValueError: too many values to unpack (expected 2)